In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

# Update this line to specify your project id
project_id = "YOUR_PROJECT_ID"

fablib = fablib_manager(project_id=project_id)

# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
#fablib.show_config()
import json
import traceback

fablib.verify_and_configure()
!bash /home/fabric/work/config_the_keys

In [ ]:
#### RUN THIS ONLY IF THE SLICE ALREADY EXISTS!
slice_name = "P4Kube" # reinitialize slize
slice = fablib.get_slice(slice_name)
#slice.list_nodes()
print('Done')

In [ ]:
# RUN THIS ONLY IF THE SLICE SHOULD BE CREATED!
slice_name = 'P4Kube' # create slize
slice = fablib.new_slice(name=slice_name)

vm_names = ['p4switch', 'master', 'client'] + [f'node{i}' for i in range(1, 11)]
for vm_name in vm_names:
    node = slice.add_node(name=vm_name, site='RUTG', image='default_ubuntu_20')
    node.set_capacities(cores=2, ram=6, disk=10)
    node.add_component(model='NIC_Basic', name='if1')

# Add additional NICs to p4switch (if2 to if12)
p4_node = slice.get_node('p4switch')
for i in range(2, 13):
    p4_node.add_component(model='NIC_Basic', name=f'if{i}')

# Add L2 bridges (ms, cs, n1s-n6s)
bridge_names = ['ms', 'cs'] + [f'n{i}s' for i in range(1, 11)]
bridges = {}
for br in bridge_names:
    bridges[br] = slice.add_l2network(name=br)

# print(slice.get_node('master').get_interfaces()[0])
# Connect master-if1 and p4switch-if1 to 'ms'
bridges['ms'].add_interface(slice.get_node('p4switch').get_interface('p4switch-if1-p1'))
bridges['ms'].add_interface(slice.get_node('master').get_interface('master-if1-p1'))

# Connect client-if1 and p4switch-if2 to 'cs'
bridges['cs'].add_interface(p4_node.get_interface('p4switch-if2-p1'))
bridges['cs'].add_interface(slice.get_node('client').get_interface('client-if1-p1'))

# Connect nodes to p4switch
for i in range(1, 11):
    node = slice.get_node(f'node{i}')
    bridges[f'n{i}s'].add_interface(node.get_interface(f'node{i}-if1-p1'))
    bridges[f'n{i}s'].add_interface(p4_node.get_interface(f'p4switch-if{i+2}-p1'))

In [ ]:
# Submit the slice
slice.submit()
print(f"Slice '{slice_name}' created and submitted.")

In [ ]:
# Changing the mac addresses of all nodes, master and client to match with the controller program
mac_map = {
    'master':  '5E:00:00:00:00:01',
    'node1':   '5E:00:00:00:00:02',
    'node2':   '5E:00:00:00:00:03',
    'node3':   '5E:00:00:00:00:04',
    'node4':   '5E:00:00:00:00:05',
    'node5':   '5E:00:00:00:00:06',
    'node6':   '5E:00:00:00:00:07',
    'node7':   '5E:00:00:00:00:08',
    'node8':   '5E:00:00:00:00:09',
    'node9':   '5E:00:00:00:00:10',
    'node10':  '5E:00:00:00:00:11',
    'client':  '5E:00:00:00:00:12'
}

# Add configuration scripts to each node
for node_name in ['master'] + [f'node{i}' for i in range(1, 11)] + ['client']:
    node = slice.get_node(node_name)
    mac = mac_map[node_name]
    if node_name == 'master':
        index = 10
    elif node_name == 'client':
        index = 21
    else:
        index = 10+int(node_name[4:])

    ip = f'{index}.0.0.2/24'
    gw = f'{index}.0.0.1'
    #iface = node.get_interfaces()[0].get_os_interface()
    iface = node.get_interface(f"{node_name}-if1-p1").get_os_interface()
    config_script = f"""
        sudo apt-get update && sudo apt-get install -y net-tools;
        sudo ifconfig {iface} down;
        sudo ip link set dev {iface} address {mac};
        sudo ifconfig {iface} {ip};
        sudo ifconfig {iface} up;
        sudo ip route add default via {gw};
    """
    node.execute(config_script)

print('Done, move to the next step')

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

p4_node = slice.get_node('p4switch')
#print(p4_node.get_management_ip())

switch_ip_map = {
    'ms': '10.0.0.1',
    'cs': '21.0.0.1',
    'n1s': '11.0.0.1',
    'n2s': '12.0.0.1',
    'n3s': '13.0.0.1',
    'n4s': '14.0.0.1',
    'n5s': '15.0.0.1',
    'n6s': '16.0.0.1',
    'n7s': '17.0.0.1',
    'n8s': '18.0.0.1',
    'n9s': '19.0.0.1',
    'n10s': '20.0.0.1'
}

# Add IPs to p4switch's interfaces according to bridge mapping
for bridge, ip in switch_ip_map.items():
    iface = p4_node.get_interface(network_name=bridge)
    #print(iface)
    subnet = f'{ip[:-1]+"0"}/24'
    #print(subnet)
    iface.ip_addr_add(addr=ip, subnet=IPv4Network(subnet))

# Boot script for switch: install tools and bring up all interfaces
# Note: IPs are already configured above so no need to set them here
switch_script = """
    sudo apt-get update && sudo apt-get install -y net-tools;
    for iface in $(ls /sys/class/net | grep enp); do
        sudo ifconfig $iface up
    done;
"""
slice.get_node('p4switch').execute(switch_script)
print('Done, move to the next step')

In [ ]:
p4_node = slice.get_node('p4switch')
switch_script = """
    git clone https://github.com/jafingerhut/p4-guide;
    sudo bash p4-guide/bin/install-p4dev-v5.sh; 
"""
slice.get_node('p4switch').execute(switch_script)
print('Done, move to the next step')

In [ ]:
# Create a token in github and clone P4Kube code
p4project = f"""
    git clone https://github.com/gareging/P4Kube.git
"""
slice.get_node('p4switch').execute(p4project)
print('Done, move to the next step')

In [ ]:
# Run compiler for P4Kube P4 program
commands = """
    pip3 install protobuf==3.20.*;
    pip3 install googleapis-common-protos;
    cd P4Kube;
    mkdir build;
    mkdir logs;
    mkdir pcaps;
    p4c-bm2-ss --p4v 16 --p4runtime-files build/p4kube.p4.p4info.txt -o build/p4kube.json p4kube.p4;
"""
slice.get_node('p4switch').execute(commands)
print('Done, move to the next step')

In [ ]:
# getting the mapping to run the command for the switch with the correct ports
p4switch = slice.get_node('p4switch')

# Port mapping: 1=master, 2=node1, ..., 11=node10, 12=client
bridge_names = ['ms'] + [f'n{i}s' for i in range(1, 11)] + ['cs']
port_map = {}

for port, bridge in enumerate(bridge_names, start=1):
    iface = p4switch.get_interface(network_name=bridge)
    linux_iface = iface.get_os_interface()
    port_map[port] = linux_iface

# Build the interface string
interface_args = ' '.join([f'--interface {port}@{iface}' for port, iface in port_map.items()])
command = f"sudo simple_switch_grpc {interface_args} build/p4kube.json --pcap pcaps --log-console"

print('Run this command in tmux to keep it in background')
print(command)

In [ ]:
# Run the controller for P4Kube P4 program
commands = """
    cd P4Kube;
    python3 mycontroller.py;
"""
slice.get_node('p4switch').execute(commands)
# Now you can test connectivity between different nodes

In [ ]:
# Installing kubernetes on master and all nodes
for node_name in ['master'] + [f'node{i}' for i in range(1, 11)]:
    node = slice.get_node(node_name)
    script = """
        # Docker install
        sudo apt update;
        sudo apt upgrade -y;
        sudo apt install docker.io -y;
        sudo systemctl enable docker;
        sudo systemctl start docker;

        # Kubernetes install
        cd /etc/apt && sudo mkdir -p keyrings && cd ~;
        echo "deb [signed-by=/etc/apt/keyrings/kubernetes-apt-keyring.gpg] https://pkgs.k8s.io/core:/stable:/v1.28/deb/ /" | sudo tee /etc/apt/sources.list.d/kubernetes.list;
        curl -fsSL https://pkgs.k8s.io/core:/stable:/v1.28/deb/Release.key | sudo gpg --dearmor -o /etc/apt/keyrings/kubernetes-apt-keyring.gpg;
        sudo apt-get update;
        sudo apt-get install -y kubelet kubeadm kubectl;
        sudo apt-mark hold kubeadm kubelet kubectl;
    """
    node.execute(script)
print('All done, move to the next step')

In [ ]:
# master setup
node = slice.get_node('master')
script = 'sudo kubeadm init --pod-network-cidr=10.244.0.0/16'
node.execute(script)
print('Update token based on the output above')

In [ ]:

JOIN_COMMAND = 'sudo ' + ''' kubeadm join UPDATE_ME_BASED_ON_OUTPUT_ABOVE '''

In [ ]:
for node_name in [f'node{i}' for i in range(1, 11)]:
    node = slice.get_node(node_name)
    node.execute(JOIN_COMMAND)
print('All done, move to the next step')

In [ ]:
# Networking for Kube
node = slice.get_node('master')
script = ''' 
    sudo mkdir -p $HOME/.kube;
    sudo cp -i /etc/kubernetes/admin.conf $HOME/.kube/config;
    sudo chown $(id -u):$(id -g) $HOME/.kube/config;
    kubectl apply -f https://docs.projectcalico.org/manifests/calico.yaml;
    '''
node.execute(script)

In [ ]:
# Clone repo
node = slice.get_node('master')
script = f'git clone https://github.com/gareging/P4Kube.git;'
node.execute(script)

In [ ]:
# Setup the deployment and service (optional)
node = slice.get_node('master')
script = '''     
    cd P4Kube/;
    kubectl create -f nginx-deployment.yaml;
    kubectl create -f nginx-service.yaml;
    kubectl patch svc ngnix-service -p '{"spec":{"externalTrafficPolicy":"Local"}}';
'''
node.execute(script)

In [ ]:
''' Metric help fix '''
for node_name in [f'node{i}' for i in range(1, 11)]:
    node = slice.get_node(node_name)
    ip = f'{10+int(node_name[4:])}.0.0.2'

    script = f'''
echo "KUBELET_EXTRA_ARGS=--node-ip={ip}" | sudo tee /etc/default/kubelet;
sudo systemctl daemon-reexec;
sudo systemctl daemon-reload;
sudo systemctl restart kubelet;
'''
    print(f"Configuring kubelet on {node_name}")
    node.execute(script)
print('All done, move to the next step')


In [ ]:
# Setup the metrics and P4Kube plugin
node = slice.get_node('master')
script = '''
    wget -c https://go.dev/dl/go1.22.0.linux-amd64.tar.gz;
    sudo tar -C /usr/local/ -xzf go1.22.0.linux-amd64.tar.gz;
    export PATH=$PATH:/usr/local/go/bin;
    cd ~/P4Kube/
    kubectl apply -f metrics-server.yaml;
    kubectl patch deployment metrics-server -n kube-system --type 'json' -p '[{"op": "add", "path": "/spec/template/spec/containers/0/args/-", "value": "--kubelet-insecure-tls"}]';
    cd ~/P4Kube/kubectl-plugin;
    go get k8s.io/client-go@latest;
    go get k8s.io/metrics/pkg/client/clientset/versioned;
    cp main.go_LOAD_AWARE main.go;
    go build;

    echo 'export PATH="/home/ubuntu/P4Kube/kubectl-plugin/:$PATH"' >> ~/.bashrc;
'''
node.execute(script)

In [ ]:
print('On master, run "kubectl p4-loadbalancer" in tmux')

In [ ]:
'''
This code will automatically replace the default html with a string "nodeX")
'''


for node_name in [f'node{i}' for i in range(1, 11)]:
    node = slice.get_node(node_name)
    script = '''
        podid=$(sudo crictl pods | awk '$0 ~ /nginx/ {print $1}');
        dockerid=$(sudo crictl ps | grep $podid | awk '{print $1}');
    '''
    script += f'''echo {node_name} | sudo crictl exec -i "$dockerid" sh -c 'cat > /usr/share/nginx/html/index.html';'''
    node.execute(script)
    print('Done on node', node_name)

In [ ]:
# INSTALL STRESS_NG on nodes
for node_name in [f'node{i}' for i in range(1, 11)]:
    node = slice.get_node(node_name)
    script = f"""
        sudo apt-get install stress-ng -y;
    """
    node.execute(script)
print('All done, move to the next step')

In [ ]:
# control the type of your experiment by updating the variables in this cell (cpu_load_test_1_node.sh or cpu_load_test_2_nodes.sh)
script_name = 'cpu_load_test_2_nodes.sh'
total = 100000
concur = 500
type_ = "LA"

# make the script executable on all nodes
for node_name in [f'node{i}' for i in range(1, 11)]:
    node = slice.get_node(node_name)
    script = f"""
        chmod +x {script_name};
    """
    node.execute(script)
print('All done, move to the next step')

In [ ]:
# RUN EXPERIMENT (STEP 1)
node = slice.get_node('master')

load_aware_string = "LOAD_AWARE" if type_ == "LA" else "NO_LOAD_AWARE"

script = "pkill -f kubectl-p4_loadbalancer"
node.execute(script)

# Build script step-by-step, capturing output after each command
script = f'''
echo "Start"
export PATH="/home/ubuntu/P4Kube/kubectl-plugin:/usr/local/go/bin:$PATH"
echo "Path installed"
cd /home/ubuntu/P4Kube/kubectl-plugin || (echo "cd failed" && exit 1)
echo "Directory switched"
echo "Stopped previous plugin run"
cp main.go_{load_aware_string} main.go || (echo "cp failed" && exit 1)
go build -o kubectl-p4_loadbalancer main.go || (echo "go build failed" && exit 1)
nohup ./kubectl-p4_loadbalancer > plugin.log 2>&1 &
echo "Plugin started"
'''
node.execute(script)
print('Enabled', type_)
print("Move to next step")


In [ ]:
# RUN EXPERIMENT (STEP 2) RUNNING THE NODE LOAD 
node_names = [f"node{i}" for i in range(1, 11)]

# Get all node objects from the slice
nodes = [slice.get_node(name) for name in node_names]

# Run the script in background with nohup (survives session end), save PID
for node in nodes:
    print(f"Starting on {node.get_name()}")
    node.execute(f'nohup bash -c "./{script_name} $HOSTNAME" > cpu.log 2>&1 & echo $! > cpu10.pid')

print(f'Starting test for {type_}..., {script_name}')
node = slice.get_node('client')
script = f'''nohup ab -n {total} -c {concur} -s 240 -l -r http://10.0.0.2/ > results_{type_}_{total}_c_{concur}.txt &'''
node.execute(script)
print('All done, move to the next step')

In [ ]:
print(type_, 'c=', concur, 'total=', total, 'test type', script_name)
node = slice.get_node('client')
node.execute(f'cat results_{type_}_{total}_c_{concur}.txt')

node_names = [f"node{i}" for i in range(1, 11)]
nodes = [slice.get_node(name) for name in node_names]
for node in nodes:
    print(f"Stopping on {node.get_name()}")
    node.execute('sudo kill $(cat cpu10.pid) || true; sudo rm -f cpu10.pid || true;')
    node.execute(f"pkill -f '/bin/bash ./{script_name}' || true")
    node.execute("sudo pkill stress-ng-cpu || true")
    #node.execute('''ps -eo pid,cmd | grep "[c]pu_10.sh" | awk '{print $1}' | xargs kill -9''')
print('CPU overload stopped')

In [ ]:
# MESAURE LOAD BALANCING SELECTION RATIO

node_names = [f"node{i}" for i in range(1, 11)]

# Get all node objects from the slice
nodes = [slice.get_node(name) for name in node_names]

# Run the script in background with nohup (survives session end), save PID
for node in nodes:
    print(f"Starting on {node.get_name()}")
    node.execute(f'nohup bash -c "./{script_name} $HOSTNAME" > cpu.log 2>&1 & echo $! > cpu10.pid')

print(f'Starting test for {type_}...')
node = slice.get_node('client')
script = f'''nohup python3 mesaure.py  > mesaure_{type_}_{script_name}.txt &'''
node.execute(script)
print('All done, move to the next step')

In [ ]:
# JUST STOPPING ALL CPU LOAD PROCESSES
node_names = [f"node{i}" for i in range(1, 11)]
nodes = [slice.get_node(name) for name in node_names]
for node in nodes:
    print(f"Stopping on {node.get_name()}")
    node.execute('sudo kill $(cat cpu10.pid) || true; sudo rm -f cpu10.pid || true;')
    node.execute(f"pkill -f '/bin/bash ./{script_name}' || true")
    node.execute("sudo pkill stress-ng-cpu || true")
    #node.execute('''ps -eo pid,cmd | grep "[c]pu_10.sh" | awk '{print $1}' | xargs kill -9''')
print('CPU overload stopped')